<a href="https://colab.research.google.com/github/Aleman778/Chatbot-Gamma/blob/master/notebooks/training_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Using TFIDF Vectors

In [1]:
!rm -r chatbot-gamma
!git clone https://github.com/Aleman778/Chatbot-Gamma chatbot-gamma
!pip install transformers
!pip install -e chatbot-gamma
import sys
sys.path.append('/content/chatbot-gamma')
import nltk
nltk.download('punkt')
nltk.download('stopwords')

rm: cannot remove 'chatbot-gamma': No such file or directory
Cloning into 'chatbot-gamma'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 94 (delta 34), reused 72 (delta 17), pack-reused 0
Unpacking objects: 100% (94/94), done.
     |████████████████████████████████| 368kB 37.5MB/s 
     |████████████████████████████████| 1.0MB 64.9MB/s 
     |████████████████████████████████| 675kB 60.3MB/s 
     |████████████████████████████████| 860kB 63.7MB/s 
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609184 sha256=768c708355bba1f6d460b64e6abef8df4ace4054df878218582805c40a4f6701
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=472b8c11a2d4d774fe52f81e11bc3a5b78ca9a680d6cb9547094ec1dc7981b04
  Stored in d

True

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from cbgamma.datasets import AmazonReviews
from cbgamma.transforms import ToTfidf
    

def train():
    train_loss = 0
    for batch_nr, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        target = to_onehot(target)
        
        prediction = network(data)
        loss = loss_function(prediction, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
        print(
            '\rEpoch {} [{}/{}] - Loss: {}'.format(
                current_epoch+1, batch_nr+1, len(train_loader), loss
            ),
            end=''
        )
        train_loss /= len(train_loader)
        losses.append(train_loss)
        
    
def validate():
    validation_loss = 0
    for batch_nr, (data, target) in enumerate(validation_loader):
        data = data.to(device)
        target = target.to(device)
        target = to_onehot(target)
        
        prediction = network(data)
        loss = loss_function(prediction, target)
        loss.backward()
        validation_loss += loss.item()
        print(
            '\rEpoch {} [{}/{}] - Validation: {}'.format(
                current_epoch+1, batch_nr+1, len(validation_loader), loss
            ),
            end=''
        )
        validation_loss /= len(validation_loader)
        validation_losses.append(validation_loss)
        
    return validation_loss


def chat():
    while True:
        text = input('>')
        if text == "exit()":
            break;
        data = train_dataset.vectorizer([text])
        data = torch.from_numpy(np.array(data)).type(torch.FloatTensor)
        data = data.to(device)
        prediction = network(data)
        sentiment = "positive" if torch.argmax(prediction) == 1 else "negative"
        print("Text: {}\nPrediction: {}\nSentiment: {}\n".format(text, prediction, sentiment))


if __name__ == "__main__":
    vectorizer = ToTfidf()
    train_dataset = AmazonReviews('./', train=True, vectorizer=vectorizer, download=True, stopwords=True)
    train_loader = DataLoader(train_dataset, batch_size=50, shuffle=False)
    
    validation_dataset = AmazonReviews('./', train=False, vectorizer=vectorizer, download=True, stopwords=True)
    validation_loader = DataLoader(validation_dataset, batch_size=50, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
    print("Training on:", device_name)
    
    network = nn.Sequential(
        nn.Linear(5199,2),
    )

    network.to(device)

    optimizer = optim.SGD(network.parameters(), lr=0.05)
    loss_function = nn.MSELoss()
    to_onehot = nn.Embedding(2, 2) 
    to_onehot.weight.data = torch.eye(2)
    to_onehot.to(device)
    epochs = 500
    current_epoch = 0
    losses = []
    validation_losses = []
    
    while current_epoch < epochs:
        train()
        loss = validate()
        current_epoch += 1

    print("\nDone training.")
    chat()

0it [00:00, ?it/s]

65536it [00:01, 57553.96it/s]            


Processing...
Processing...
Done!
Training on: Tesla T4
Epoch 500 [2/2] - Validation: 0.04709818586707115
Done training.
>This is a good product, can strongly recommend it
Text: This is a good product, can strongly recommend it
Prediction: tensor([[-0.1503,  1.1517]], device='cuda:0', grad_fn=<AddmmBackward>)
Sentiment: positive

>There is something wrong with my product i received
Text: There is something wrong with my product i received
Prediction: tensor([[0.6407, 0.3781]], device='cuda:0', grad_fn=<AddmmBackward>)
Sentiment: negative

>Test1234
Text: Test1234
Prediction: tensor([[0.5865, 0.4090]], device='cuda:0', grad_fn=<AddmmBackward>)
Sentiment: negative

>Apples
Text: Apples
Prediction: tensor([[0.5865, 0.4090]], device='cuda:0', grad_fn=<AddmmBackward>)
Sentiment: negative

>Bad
Text: Bad
Prediction: tensor([[ 1.1641, -0.1576]], device='cuda:0', grad_fn=<AddmmBackward>)
Sentiment: negative

>Good
Text: Good
Prediction: tensor([[-0.3495,  1.3492]], device='cuda:0', grad_fn=<Ad